In [27]:
#loading libraries
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

In [28]:
tennis_data <- read_csv("tennis_data.csv", show_col_types = FALSE)

New names:
• `` -> `...1`


In [64]:
#prepping data
set.seed(4673)

winner_data <- tennis_data |> select(winner_rank_points, winner_ht, w_df, w_bpSaved, w_bpFaced) |> 
    filter(w_bpSaved != 0) |> filter(w_bpFaced != 0) |> mutate(bpratio = w_bpSaved/w_bpFaced) |> na.omit() |>
    rename(rank_points = winner_rank_points, height = winner_ht, double_faults = w_df) |>
    select(-w_bpSaved, -w_bpFaced)
loser_data <- tennis_data |> select(loser_rank_points, loser_ht, l_df, l_bpSaved, l_bpFaced) |> 
    filter(l_bpSaved != 0) |> filter(l_bpFaced != 0) |> mutate(bpratio = l_bpSaved/l_bpFaced) |> na.omit() |>
    rename(rank_points = loser_rank_points, height = loser_ht, double_faults = l_df) |>
    select(-l_bpSaved, -l_bpFaced)

player_data <- rbind(winner_data, loser_data)

In [67]:
#linearizing data
player_data_l <- player_data |>
    summarize(player_data, mean_ht = mean(height)) |>
    mutate(bp_percent = bpratio*100)
player_data_l

rank_points,height,double_faults,bpratio,mean_ht,bp_percent
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3590,178,3,0.5,186.156,50
3590,178,2,1.0,186.156,100
200,188,2,0.8,186.156,80
⋮,⋮,⋮,⋮,⋮,⋮
667,178,2,0.4545455,186.156,45.45455
2320,188,3,0.7000000,186.156,70.00000
667,178,0,0.3636364,186.156,36.36364


In [68]:
#splitting data
player_split <- initial_split(player_data_l, prop = .75, strata = rank_points)  
player_train <- training(player_split)
player_test <- testing(player_split)

In [72]:
#model
lm_spec <- linear_reg() |>
    set_engine("lm") |>
    set_mode("regression")

#recipe
player_recipe <- recipe(rank_points ~ height + bp_percent + double_faults, data = player_train)


workflow <- workflow() |>
    add_recipe(player_recipe) |>
    add_model(lm_spec) |>
    fit(player_train)
workflow

══ Workflow [trained] ══════════════════════════════════════════════════════════
Preprocessor: Recipe
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────
0 Recipe Steps

── Model ───────────────────────────────────────────────────────────────────────

Call:
stats::lm(formula = ..y ~ ., data = data)

Coefficients:
  (Intercept)         height     bp_percent  double_faults  
    -2839.938         22.263          8.226        -35.457  
